In [ ]:
import os
import torch
from sklearn import metrics
import logging
from timeit import default_timer as timer
import pandas as pn

In [2]:
from context import resnet
from context import utils
from context import data_loader
from context import data_utils

In [16]:
BATCH_SIZE = 2
STATE_DICT_PATH = '../state'
DATA_PATH = '../data/imagenet_red'
STORE_RESULT_PATH = ''

In [4]:
def evaluateModel(model, loader, classes, batch_size): 
    pred, grndT = [], []
    for (images, labels) in iter(loader):
        if torch.cuda.is_available():
            images = images.cuda(non_blocking=True)
            labels = labels.cuda(non_blocking=True)
            
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        pred = pred + [classes[predicted[k]] for k in range(min(batch_size, labels.shape[0]))]
        grndT = grndT + [classes[labels[j]] for j in range(min(batch_size, labels.shape[0]))]
    return grndT, pred

def timed_decorator(function):
    def inner(*args, **kwargs):
        start = timer()
        result = function(*args, **kwargs)
        end = timer()
        print(f'{(end - start) * 1000} [ms]')
        return result
    return inner

@timed_decorator
def executeQualityBench(arch_name: str, loader, skip_n: int, classes, batch_size: int):
    model = utils.getModelWithOptimized(arch_name, n=skip_n, batch_size=batch_size)

    if torch.cuda.is_available():
        model = torch.nn.DataParallel(model).cuda()
    model.eval()
    arch = arch_name.split('-')[0]
    model, _, _, _ = utils.resumeFromPath(os.path.join(STATE_DICT_PATH, f'{arch}_model_best.pth.tar'), model)
    grndT, pred = evaluateModel(model, loader, classes, batch_size)

    #logging.debug(metrics.classification_report(grndT, pred, digits=3))
    #acc = metrics.accuracy_score(grndT, pred)
    #prec = metrics.precision_score(grndT, pred, average='macro')
    #rec = metrics.recall_score(grndT, pred, average='macro')
    #f1 = metrics.f1_score(grndT, pred, average='macro')
    return grndT, pred

In [5]:
experiment_data = list()

logging.basicConfig(level=logging.INFO)

model_name = 'resnet50-drop-rand-n'

skip_layers = 4
classes = data_utils.getLabelToClassMapping(DATA_PATH)
_, _, loader = data_loader.get_zipped_dataloaders(DATA_PATH, batch_size=BATCH_SIZE, use_valid=True)

for run in range(0, 30):
    print(f'Executing Run {run + 1} of 30')
    model = utils.getModelWithOptimized(model_name, n=skip_layers, batch_size=BATCH_SIZE)
    groundT, pred = executeQualityBench(model_name, loader, skip_n=skip_layers, classes=classes, batch_size=BATCH_SIZE)
    policy = resnet.DropPolicies.getDropPolicy()
    layer_config = []
    if isinstance(policy, resnet.DropPolicies.ResNetDropRandNPolicy):
        layer_config = policy.getDropConfig()
    experiment_data.append({
        'skip_n': skip_layers,
        'ground_truth': groundT, 
        'prediction': pred, 
        'configuration': layer_config,
        'acc': metrics.accuracy_score(groundT, pred)
        })

df = pn.DataFrame(experiment_data)
df.to_csv(os.path.join(STORE_RESULT_PATH, 'resnet50_experiment_results.csv'), index=False)

def print_data(data):
    for datum in data:
        print()
        print(datum['skip_n'])
        print(len(datum['ground_truth']))
        print(len(datum['prediction']))
        print(len(datum['configuration']))
        print()





../data/imagenet_red/index-val.txt
../data/imagenet_red/index-train.txt
../data/imagenet_red/index-val.txt
Executing Run 1 of 30
INFO:root:=> loaded checkpoint '../state/resnet50_model_best.pth.tar' (epoch 86 -> 87)
326329.661336 [ms]
Executing Run 2 of 30
INFO:root:=> loaded checkpoint '../state/resnet50_model_best.pth.tar' (epoch 86 -> 87)
321259.4235119999 [ms]
Executing Run 3 of 30
INFO:root:=> loaded checkpoint '../state/resnet50_model_best.pth.tar' (epoch 86 -> 87)
351528.04596199986 [ms]
Executing Run 4 of 30
INFO:root:=> loaded checkpoint '../state/resnet50_model_best.pth.tar' (epoch 86 -> 87)
307732.3981750001 [ms]
Executing Run 5 of 30
INFO:root:=> loaded checkpoint '../state/resnet50_model_best.pth.tar' (epoch 86 -> 87)
328355.285827 [ms]
Executing Run 6 of 30
INFO:root:=> loaded checkpoint '../state/resnet50_model_best.pth.tar' (epoch 86 -> 87)
348431.66122499976 [ms]
Executing Run 7 of 30
INFO:root:=> loaded checkpoint '../state/resnet50_model_best.pth.tar' (epoch 86 -> 87

KeyboardInterrupt: 

In [19]:
temp = list()

temp.append({
    'skip_n': 1,
    'ground_truth': ['test2', 'test3', 'test3'], 
    'prediction': ['test1', 'test2', 'test3'], 
    'configuration': [True, False, False, True],
    'acc': 0.76
})
temp.append({
    'skip_n': 3,
    'ground_truth': ['test1', 'test1', 'test1'], 
    'prediction': ['test2', 'test2', 'test3'], 
    'configuration': [True, True, False, True],
    'acc': 0.70
})
temp.append({
    'skip_n': 1,
    'ground_truth': ['test3', 'test1', 'test3'], 
    'prediction': ['test1', 'test1', 'test3'], 
    'configuration': [False, False, False, False],
    'acc': 0.22
})


df = pn.DataFrame(temp)
df.to_csv(os.path.join(STORE_RESULT_PATH, 'resnet50_experiment_results.csv'), index=False)
df = pn.read_csv(os.path.join(STORE_RESULT_PATH, 'resnet50_experiment_results.csv'))

,skip_n,ground_truth,prediction,configuration,acc
0,1,"['test2', 'test3', 'test3']","['test1', 'test2', 'test3']","[True, False, False, True]",0.76
1,3,"['test1', 'test1', 'test1']","['test2', 'test2', 'test3']","[True, True, False, True]",0.70
2,1,"['test3', 'test1', 'test3']","['test1', 'test1', 'test3']","[False, False, False, False]",0.22
